# 🚀 해외직구 사업 초기분석 도구

해외 쇼핑몰 상품을 국내에 판매하는 사업의 수익성을 분석하는 종합 도구입니다.

## 📊 주요 기능
1. **해외 쇼핑몰 데이터 수집** (Amazon, AliExpress 등)
2. **국내 마켓 가격 분석** (쿠팡, 11번가 등)
3. **트렌드 분석** (Google Trends, 네이버 쇼핑)
4. **수익성 계산** (마진율, ROI 예측)
5. **종합 점수 산출** (투자 우선순위 결정)

In [ ]:
# 필요한 라이브러리 설치 및 임포트
!pip install requests beautifulsoup4 pandas numpy matplotlib seaborn plotly pytrends selenium webdriver-manager

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from bs4 import BeautifulSoup
from urllib.parse import urljoin, quote
from datetime import datetime, timedelta
import time
import json
import re
import warnings
from pytrends.request import TrendReq
from IPython.display import display, HTML
import logging

# 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.style.use('seaborn-v0_8')
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)

print("📦 라이브러리 로딩 완료!")
print("🔧 분석 도구 초기화 중...")

In [ ]:
class MarketAnalyzer:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'MarketAnalyzer/1.0 (Business Analysis Tool)',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive'
        })
        
        # 환율 정보 (실제로는 API에서 가져와야 함)
        self.exchange_rates = {
            'USD': 1350,  # 1 USD = 1350 KRW
            'CNY': 185,   # 1 CNY = 185 KRW
            'EUR': 1450,  # 1 EUR = 1450 KRW
        }
        
        # 비용 구조
        self.cost_structure = {
            'shipping_rate': 0.15,      # 배송비 15%
            'platform_fee': 0.05,       # 플랫폼 수수료 5%
            'payment_fee': 0.03,        # 결제 수수료 3%
            'tax_rate': 0.1,           # 세금 10%
            'operation_cost': 0.05,     # 운영비 5%
            'target_margin': 0.2        # 목표 마진 20%
        }
    
    def search_overseas_products(self, keyword, source='aliexpress', limit=10):
        """해외 쇼핑몰에서 상품 검색"""
        products = []
        
        if source == 'aliexpress':
            # AliExpress 검색 시뮬레이션 (실제로는 API 또는 크롤링)
            # 데모용 샘플 데이터
            sample_products = [
                {
                    'title': f'{keyword} Premium Quality Item {i+1}',
                    'price': round(np.random.uniform(5, 50), 2),
                    'currency': 'USD',
                    'rating': round(np.random.uniform(4.0, 4.9), 1),
                    'reviews': np.random.randint(10, 1000),
                    'orders': np.random.randint(50, 2000),
                    'shipping_time': f'{np.random.randint(7, 21)} days',
                    'source': 'AliExpress',
                    'url': f'https://aliexpress.com/item/{np.random.randint(100000, 999999)}.html'
                }
                for i in range(limit)
            ]
            products.extend(sample_products)
            
        return pd.DataFrame(products)
    
    def search_domestic_market(self, keyword, platform='coupang', limit=10):
        """국내 마켓에서 경쟁상품 검색"""
        products = []
        
        if platform == 'coupang':
            # 쿠팡 검색 시뮬레이션
            sample_products = [
                {
                    'title': f'{keyword} 국내 판매 상품 {i+1}',
                    'price': round(np.random.uniform(15000, 80000), -2),
                    'currency': 'KRW',
                    'rating': round(np.random.uniform(3.5, 4.8), 1),
                    'reviews': np.random.randint(5, 500),
                    'delivery': '로켓배송' if np.random.random() > 0.3 else '일반배송',
                    'seller_type': '쿠팡' if np.random.random() > 0.5 else '개별판매자',
                    'platform': 'Coupang',
                    'url': f'https://coupang.com/vp/products/{np.random.randint(100000, 999999)}'
                }
                for i in range(limit)
            ]
            products.extend(sample_products)
            
        return pd.DataFrame(products)
    
    def get_trend_data(self, keyword, timeframe='today 12-m', geo='KR'):
        """Google Trends 데이터 조회"""
        try:
            # Google Trends 초기화
            pytrends = TrendReq(hl='ko-KR', tz=540)
            
            # 키워드 설정
            pytrends.build_payload([keyword], cat=0, timeframe=timeframe, geo=geo)
            
            # 시계열 데이터
            interest_over_time = pytrends.interest_over_time()
            
            # 관련 검색어
            related_queries = pytrends.related_queries()
            
            return {
                'interest_over_time': interest_over_time,
                'related_queries': related_queries,
                'current_trend': interest_over_time[keyword].iloc[-1] if not interest_over_time.empty else 0
            }
            
        except Exception as e:
            print(f"트렌드 데이터 조회 실패: {e}")
            # 샘플 데이터 반환
            dates = pd.date_range(end=datetime.now(), periods=12, freq='M')
            trend_values = np.random.randint(20, 100, 12)
            
            return {
                'interest_over_time': pd.DataFrame({
                    keyword: trend_values
                }, index=dates),
                'related_queries': {'top': pd.DataFrame(), 'rising': pd.DataFrame()},
                'current_trend': trend_values[-1]
            }
    
    def calculate_profit_margin(self, overseas_price, currency='USD', domestic_price=None):
        """수익성 계산"""
        # 원화 환산
        krw_cost = overseas_price * self.exchange_rates[currency]
        
        # 총 비용 계산
        shipping_cost = krw_cost * self.cost_structure['shipping_rate']
        platform_fee = krw_cost * self.cost_structure['platform_fee']
        payment_fee = krw_cost * self.cost_structure['payment_fee']
        tax = krw_cost * self.cost_structure['tax_rate']
        operation_cost = krw_cost * self.cost_structure['operation_cost']
        
        total_cost = krw_cost + shipping_cost + platform_fee + payment_fee + tax + operation_cost
        
        # 목표 판매가 계산
        target_selling_price = total_cost * (1 + self.cost_structure['target_margin'])
        
        # 경쟁가격과 비교
        if domestic_price:
            price_competitiveness = (domestic_price - target_selling_price) / domestic_price
            margin_at_market_price = (domestic_price - total_cost) / domestic_price
        else:
            price_competitiveness = None
            margin_at_market_price = self.cost_structure['target_margin']
        
        return {
            'original_cost_krw': krw_cost,
            'total_cost': total_cost,
            'target_selling_price': target_selling_price,
            'margin_at_target': self.cost_structure['target_margin'],
            'margin_at_market': margin_at_market_price,
            'price_competitiveness': price_competitiveness,
            'cost_breakdown': {
                'product_cost': krw_cost,
                'shipping': shipping_cost,
                'platform_fee': platform_fee,
                'payment_fee': payment_fee,
                'tax': tax,
                'operation': operation_cost
            }
        }
    
    def calculate_opportunity_score(self, overseas_product, domestic_products, trend_data):
        """기회 점수 계산"""
        # 기본 점수들
        scores = {
            'demand_score': 0,      # 수요 점수 (0-100)
            'competition_score': 0,  # 경쟁 점수 (낮을수록 좋음)
            'margin_score': 0,      # 마진 점수 (0-100)
            'trend_score': 0,       # 트렌드 점수 (0-100)
            'quality_score': 0      # 품질 점수 (0-100)
        }
        
        # 수요 점수 (해외 주문량 기준)
        if overseas_product['orders'] > 1000:
            scores['demand_score'] = 90
        elif overseas_product['orders'] > 500:
            scores['demand_score'] = 70
        elif overseas_product['orders'] > 100:
            scores['demand_score'] = 50
        else:
            scores['demand_score'] = 30
        
        # 경쟁 점수 (국내 유사상품 수)
        competition_count = len(domestic_products)
        if competition_count < 5:
            scores['competition_score'] = 90  # 경쟁 적음 = 좋음
        elif competition_count < 15:
            scores['competition_score'] = 60
        else:
            scores['competition_score'] = 30
        
        # 마진 점수
        if domestic_products is not None and len(domestic_products) > 0:
            avg_domestic_price = domestic_products['price'].mean()
            profit_calc = self.calculate_profit_margin(
                overseas_product['price'], 
                overseas_product['currency'],
                avg_domestic_price
            )
            
            margin = profit_calc['margin_at_market']
            if margin > 0.3:  # 30% 이상 마진
                scores['margin_score'] = 90
            elif margin > 0.2:  # 20% 이상 마진
                scores['margin_score'] = 70
            elif margin > 0.1:  # 10% 이상 마진
                scores['margin_score'] = 50
            else:
                scores['margin_score'] = 20
        
        # 트렌드 점수
        scores['trend_score'] = min(trend_data['current_trend'], 100)
        
        # 품질 점수 (평점 및 리뷰 수)
        rating_score = (overseas_product['rating'] / 5.0) * 50
        review_score = min(overseas_product['reviews'] / 100 * 50, 50)
        scores['quality_score'] = rating_score + review_score
        
        # 가중 평균으로 종합 점수 계산
        weights = {
            'demand_score': 0.25,
            'competition_score': 0.2,
            'margin_score': 0.25,
            'trend_score': 0.15,
            'quality_score': 0.15
        }
        
        total_score = sum(scores[key] * weights[key] for key in scores.keys())
        
        return {
            'total_score': round(total_score, 1),
            'detailed_scores': scores,
            'recommendation': self._get_recommendation(total_score)
        }
    
    def _get_recommendation(self, score):
        """점수 기반 추천 등급"""
        if score >= 80:
            return {'level': 'EXCELLENT', 'action': '즉시 진행 추천', 'color': 'green'}
        elif score >= 65:
            return {'level': 'GOOD', 'action': '진행 고려', 'color': 'blue'}
        elif score >= 50:
            return {'level': 'AVERAGE', 'action': '추가 분석 필요', 'color': 'orange'}
        else:
            return {'level': 'POOR', 'action': '진행 비추천', 'color': 'red'}

# 분석기 초기화
analyzer = MarketAnalyzer()
print("✅ MarketAnalyzer 초기화 완료!")
print("🚀 분석 준비 완료!")

In [ ]:
# 📊 실제 분석 시작
# 분석하고 싶은 상품 키워드 입력
KEYWORD = "wireless earbuds"  # 예시: 무선이어폰
print(f"🔍 '{KEYWORD}' 상품 분석 시작...")

# 1. 해외 상품 데이터 수집
print("\n1️⃣ 해외 쇼핑몰 상품 검색 중...")
overseas_products = analyzer.search_overseas_products(KEYWORD, limit=15)
print(f"   ✅ {len(overseas_products)}개 해외 상품 발견")

# 2. 국내 경쟁상품 데이터 수집
print("\n2️⃣ 국내 마켓 경쟁상품 검색 중...")
domestic_products = analyzer.search_domestic_market(KEYWORD, limit=15)
print(f"   ✅ {len(domestic_products)}개 국내 상품 발견")

# 3. 트렌드 데이터 수집
print("\n3️⃣ Google Trends 데이터 수집 중...")
trend_data = analyzer.get_trend_data(KEYWORD)
print(f"   ✅ 현재 트렌드 지수: {trend_data['current_trend']}")

print("\n📊 기초 데이터 수집 완료!")

In [ ]:
# 📈 해외 상품 분석 및 시각화
print("📊 해외 상품 데이터 분석")
print("="*50)

# 기본 통계
print(f"평균 가격: ${overseas_products['price'].mean():.2f}")
print(f"가격 범위: ${overseas_products['price'].min():.2f} ~ ${overseas_products['price'].max():.2f}")
print(f"평균 평점: {overseas_products['rating'].mean():.1f}/5.0")
print(f"총 주문량: {overseas_products['orders'].sum():,}건")

# 상위 5개 상품 표시
print("\n🏆 주문량 TOP 5 상품:")
top_products = overseas_products.nlargest(5, 'orders')[['title', 'price', 'rating', 'orders']]
display(top_products)

# 가격 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 가격 히스토그램
axes[0].hist(overseas_products['price'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Price (USD)')
axes[0].set_ylabel('Count')
axes[0].set_title('해외 상품 가격 분포')

# 가격 vs 주문량 산점도
scatter = axes[1].scatter(overseas_products['price'], overseas_products['orders'], 
                         c=overseas_products['rating'], cmap='viridis', alpha=0.6)
axes[1].set_xlabel('Price (USD)')
axes[1].set_ylabel('Orders')
axes[1].set_title('가격 vs 주문량 (색상: 평점)')
plt.colorbar(scatter, ax=axes[1], label='Rating')

plt.tight_layout()
plt.show()

In [ ]:
# 💰 수익성 분석
print("💰 수익성 분석")
print("="*50)

# 각 해외 상품에 대한 수익성 계산
profit_analysis = []

avg_domestic_price = domestic_products['price'].mean()
print(f"국내 평균 판매가: {avg_domestic_price:,.0f}원")

for idx, product in overseas_products.iterrows():
    profit_calc = analyzer.calculate_profit_margin(
        product['price'], 
        product['currency'],
        avg_domestic_price
    )
    
    profit_analysis.append({
        'product_title': product['title'][:30] + "...",
        'overseas_price_usd': product['price'],
        'total_cost_krw': profit_calc['total_cost'],
        'target_price_krw': profit_calc['target_selling_price'],
        'market_price_krw': avg_domestic_price,
        'margin_at_market': profit_calc['margin_at_market'],
        'price_competitiveness': profit_calc['price_competitiveness'],
        'orders': product['orders'],
        'rating': product['rating']
    })

profit_df = pd.DataFrame(profit_analysis)

# 수익성이 좋은 상위 5개 상품
print("\n🎯 마진율 TOP 5 상품:")
top_margin_products = profit_df.nlargest(5, 'margin_at_market')[[
    'product_title', 'overseas_price_usd', 'margin_at_market', 'orders', 'rating'
]]
top_margin_products['margin_at_market'] = top_margin_products['margin_at_market'].apply(lambda x: f"{x:.1%}")
display(top_margin_products)

# 수익성 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 마진율 히스토그램
axes[0].hist(profit_df['margin_at_market'].dropna() * 100, bins=10, alpha=0.7, color='lightgreen')
axes[0].set_xlabel('Margin Rate (%)')
axes[0].set_ylabel('Count')
axes[0].set_title('마진율 분포')
axes[0].axvline(20, color='red', linestyle='--', label='목표 마진 20%')
axes[0].legend()

# 마진율 vs 주문량
valid_data = profit_df.dropna(subset=['margin_at_market'])
scatter = axes[1].scatter(valid_data['margin_at_market'] * 100, valid_data['orders'],
                         c=valid_data['rating'], cmap='RdYlBu_r', alpha=0.6, s=60)
axes[1].set_xlabel('Margin Rate (%)')
axes[1].set_ylabel('Orders')
axes[1].set_title('마진율 vs 주문량 (색상: 평점)')
axes[1].axvline(20, color='red', linestyle='--', alpha=0.7, label='목표 마진 20%')
axes[1].legend()
plt.colorbar(scatter, ax=axes[1], label='Rating')

plt.tight_layout()
plt.show()

In [ ]:
# 📊 트렌드 분석
print("📊 시장 트렌드 분석")
print("="*50)

# 트렌드 데이터 시각화
if not trend_data['interest_over_time'].empty:
    plt.figure(figsize=(12, 6))
    
    trend_df = trend_data['interest_over_time']
    plt.plot(trend_df.index, trend_df[KEYWORD], marker='o', linewidth=2, markersize=6)
    plt.title(f"'{KEYWORD}' Google 검색 트렌드 (최근 12개월)")
    plt.xlabel('날짜')
    plt.ylabel('검색 관심도')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    
    # 현재 트렌드 상태
    current_value = trend_df[KEYWORD].iloc[-1]
    prev_value = trend_df[KEYWORD].iloc[-6] if len(trend_df) > 6 else trend_df[KEYWORD].iloc[0]
    trend_change = ((current_value - prev_value) / prev_value * 100) if prev_value != 0 else 0
    
    plt.axhline(current_value, color='red', linestyle='--', alpha=0.7, 
               label=f'현재: {current_value} ({trend_change:+.1f}%)')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📈 트렌드 분석 결과:")
    print(f"   현재 관심도: {current_value}/100")
    print(f"   6개월 대비 변화: {trend_change:+.1f}%")
    
    if trend_change > 20:
        print("   🚀 상승 트렌드! 진입 시기 좋음")
    elif trend_change > 0:
        print("   📈 완만한 상승세")
    elif trend_change > -20:
        print("   📉 완만한 하락세")
    else:
        print("   ⚠️ 하락 트렌드, 주의 필요")
else:
    print("트렌드 데이터를 불러올 수 없습니다.")

In [ ]:
# 🏆 종합 기회 점수 분석
print("🏆 종합 기회 점수 분석")
print("="*50)

# 각 상품의 기회 점수 계산
opportunity_analysis = []

for idx, product in overseas_products.iterrows():
    score_result = analyzer.calculate_opportunity_score(
        product.to_dict(),
        domestic_products,
        trend_data
    )
    
    opportunity_analysis.append({
        'product_title': product['title'][:40] + "...",
        'price_usd': product['price'],
        'orders': product['orders'],
        'rating': product['rating'],
        'total_score': score_result['total_score'],
        'recommendation': score_result['recommendation']['level'],
        'action': score_result['recommendation']['action'],
        'demand_score': score_result['detailed_scores']['demand_score'],
        'margin_score': score_result['detailed_scores']['margin_score'],
        'competition_score': score_result['detailed_scores']['competition_score'],
        'trend_score': score_result['detailed_scores']['trend_score'],
        'quality_score': score_result['detailed_scores']['quality_score']
    })

opportunity_df = pd.DataFrame(opportunity_analysis)

# TOP 10 기회 상품
print("\n🎯 투자 우선순위 TOP 10:")
top_opportunities = opportunity_df.nlargest(10, 'total_score')[[
    'product_title', 'price_usd', 'total_score', 'recommendation', 'action'
]]
display(top_opportunities)

# 추천 등급별 분포
recommendation_counts = opportunity_df['recommendation'].value_counts()
print(f"\n📊 추천 등급별 분포:")
for level, count in recommendation_counts.items():
    print(f"   {level}: {count}개 ({count/len(opportunity_df)*100:.1f}%)")

In [ ]:
# 📊 종합 대시보드
print("📊 종합 분석 대시보드")
print("="*70)

# 레이더 차트로 상위 5개 상품 비교
top_5_products = opportunity_df.nlargest(5, 'total_score')

# Plotly 레이더 차트
categories = ['Demand', 'Margin', 'Competition', 'Trend', 'Quality']

fig = go.Figure()

colors = ['red', 'blue', 'green', 'orange', 'purple']

for i, (idx, product) in enumerate(top_5_products.head().iterrows()):
    values = [
        product['demand_score'],
        product['margin_score'], 
        product['competition_score'],
        product['trend_score'],
        product['quality_score']
    ]
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories,
        fill='toself',
        name=f"Product {i+1} (Score: {product['total_score']:.1f})",
        line=dict(color=colors[i])
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 100]
        )),
    showlegend=True,
    title="TOP 5 상품 종합 점수 비교",
    width=800,
    height=600
)

fig.show()

# 최종 요약
print("\n🎯 최종 분석 요약")
print("="*50)

excellent_products = len(opportunity_df[opportunity_df['recommendation'] == 'EXCELLENT'])
good_products = len(opportunity_df[opportunity_df['recommendation'] == 'GOOD'])
total_products = len(opportunity_df)

print(f"📈 분석 상품 수: {total_products}개")
print(f"🌟 즉시 진행 추천: {excellent_products}개 ({excellent_products/total_products*100:.1f}%)")
print(f"✅ 진행 고려 대상: {good_products}개 ({good_products/total_products*100:.1f}%)")

if excellent_products > 0:
    best_product = opportunity_df.loc[opportunity_df['total_score'].idxmax()]
    print(f"\n🏆 최고 추천 상품:")
    print(f"   제품: {best_product['product_title']}")
    print(f"   가격: ${best_product['price_usd']:.2f}")
    print(f"   종합점수: {best_product['total_score']:.1f}/100")
    print(f"   추천사유: {best_product['action']}")

avg_score = opportunity_df['total_score'].mean()
print(f"\n📊 '{KEYWORD}' 카테고리 전체 평가: {avg_score:.1f}/100점")

if avg_score >= 70:
    print("🚀 매우 유망한 사업 영역입니다!")
elif avg_score >= 50:
    print("📈 적정한 수준의 사업 기회가 있습니다.")
else:
    print("⚠️ 진입 전 추가적인 시장 조사가 필요합니다.")

In [ ]:
# 💾 결과 저장
print("💾 분석 결과 저장")
print("="*30)

# 결과를 엑셀 파일로 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"market_analysis_{KEYWORD.replace(' ', '_')}_{timestamp}.xlsx"

with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    # 해외 상품 데이터
    overseas_products.to_excel(writer, sheet_name='해외상품', index=False)
    
    # 국내 상품 데이터  
    domestic_products.to_excel(writer, sheet_name='국내상품', index=False)
    
    # 수익성 분석
    profit_df.to_excel(writer, sheet_name='수익성분석', index=False)
    
    # 기회 점수 분석
    opportunity_df.to_excel(writer, sheet_name='기회점수', index=False)
    
    # 트렌드 데이터
    if not trend_data['interest_over_time'].empty:
        trend_data['interest_over_time'].to_excel(writer, sheet_name='트렌드데이터')

print(f"✅ 분석 결과가 '{filename}'에 저장되었습니다.")
print("\n📋 저장된 시트:")
print("   - 해외상품: 수집된 해외 상품 정보")
print("   - 국내상품: 국내 경쟁 상품 정보")
print("   - 수익성분석: 마진율 및 비용 구조 분석")
print("   - 기회점수: 종합 투자 우선순위")
print("   - 트렌드데이터: Google Trends 시계열 데이터")

print("\n🎉 분석 완료! 비즈니스 의사결정에 활용하세요.")